# Start: Analysiere Daten

In [321]:
import csv
import re
import pandas as pd

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush (3).csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)
    return data


df_queried_2 = df_queried[df_queried['costs'] == str(1)]
data = add_data(data, df_queried_2)
data = data.reset_index().astype('Float32')
# print(len(data))


#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'Pressure cylinder']]

print(len(x))
print(len(y))


137
137


In [335]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectFromModel


# Daten in Trainings- und Testdatensatz aufteilen 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Hyperparameter-Tuning mit Grid-Suche für Random Forest Regressor 
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2],
    'max_features': ['log2','sqrt'],
    'bootstrap': [True, False],
    # 'criterion': ['mse', 'mae'],
}

rf_model = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error') 
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Beste Hyperparameter: {best_params}")

# Modell trainieren und bewerten
best_rf_model = RandomForestRegressor(**best_params, random_state=42) 
best_rf_model.fit(X_train, y_train)

# Vorhersagen machen mit dem Random Forest Modell
y_pred_rf = best_rf_model.predict(X_test)

# Feature Selection basierend auf den RF Modell
selector = SelectFromModel(best_rf_model, threshold='mean', prefit=True)

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Modell trainieren erneut trainieren und bewerten 
best_rf_model.fit(X_train_selected, y_train)
y_pred_rf = best_rf_model.predict(X_test_selected)

# Modellbewertung für Random Forest
# mse_rf = mean_squared_error(y_test, y_pred_rf) 
# print(f"Mean Squared Error (Random Forest): {mse_rf}") 
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf) 
print(f"Mean Absolute Percentage Error (Random Forest): {100-mape_rf*100}")

# MAPE für jeden Ausgabeparameter nach Random Forest 
mape_per_output_rf = mean_absolute_percentage_error(y_test, y_pred_rf, multioutput='raw_values') 
for i, mape in enumerate(mape_per_output_rf):
    print(f"MAPE für Ausgabeparameter {i+1} (Random Forest): {100-mape*100}")




Beste Hyperparameter: {'bootstrap': False, 'max_depth': 7, 'max_features': 'log2', 'max_leaf_nodes': 30, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100}
Mean Absolute Percentage Error (Random Forest): 65.28433345541711
MAPE für Ausgabeparameter 1 (Random Forest): 66.00588699755795
MAPE für Ausgabeparameter 2 (Random Forest): 29.760751682022217
MAPE für Ausgabeparameter 3 (Random Forest): 73.09039332494481
MAPE für Ausgabeparameter 4 (Random Forest): 92.28030181714348


c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
